In [1]:
# Using R, load the Titanic dataset. 
# Remove any duplicates from the table.
# Convert the gender column into a numeric field. 
# Create a column that gives total number of relatives on board.
# Then create a regression that explains the # of people that survive based on the available data.

In [2]:
library("XLConnect")

Loading required package: XLConnectJars
XLConnect 0.2-13 by Mirai Solutions GmbH [aut],
  Martin Studer [cre],
  The Apache Software Foundation [ctb, cph] (Apache POI),
  Graph Builder [ctb, cph] (Curvesapi Java library)
http://www.mirai-solutions.com ,
http://miraisolutions.wordpress.com


In [3]:
df <- readWorksheetFromFile("titanic3.xls",
                           sheet=1)
head(df,10)

pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NA,"St Louis, MO"
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NA,"Montreal, PQ / Chesterville, ON"
1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NA,NA,"Montreal, PQ / Chesterville, ON"
1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NA,135,"Montreal, PQ / Chesterville, ON"
1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NA,NA,"Montreal, PQ / Chesterville, ON"
1,1,"Anderson, Mr. Harry",male,48.0000,0,0,19952,26.5500,E12,S,3,NA,"New York, NY"
1,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,13502,77.9583,D7,S,10,NA,"Hudson, NY"
1,0,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,112050,0.0000,A36,S,NA,NA,"Belfast, NI"
1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,11769,51.4792,C101,S,D,NA,"Bayside, Queens, NY"
1,0,"Artagaveytia, Mr. Ramon",male,71.0000,0,0,PC 17609,49.5042,NA,C,NA,22,"Montevideo, Uruguay"


In [4]:
# Remove any duplicates from the table:

In [5]:
anyDuplicated(df)

[1] 0

In [6]:
df[duplicated(df),]

pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest


In [7]:
# Did not find any duplicate rows. However, there could be duplicate entries for the same person but with minor differences.
# Ticket # is not a good way to look for duplicates as multiple people traveling together could travel on the same ticket.
# Therefore tried checking for duplicate names.

In [8]:
anyDuplicated(df$name)
# There is at least one duplicated name. The first duplicate is at index 727.

[1] 727

In [9]:
df[duplicated(df$name),]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
727,3,0,"Connolly, Miss. Kate",female,30,0,0,330972,7.6292,NA,Q,NA,NA,Ireland
926,3,0,"Kelly, Mr. James",male,44,0,0,363592,8.0500,NA,S,NA,NA,NA


In [10]:
df[df$name == 'Connolly, Miss. Kate',]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
726,3,1,"Connolly, Miss. Kate",female,22,0,0,370373,7.7500,NA,Q,13,NA,Ireland
727,3,0,"Connolly, Miss. Kate",female,30,0,0,330972,7.6292,NA,Q,NA,NA,Ireland


In [11]:
df[df$name == 'Kelly, Mr. James',]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
925,3,0,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NA,Q,NA,70,NA
926,3,0,"Kelly, Mr. James",male,44.0,0,0,363592,8.0500,NA,S,NA,NA,NA


In [12]:
# There are 2 female passengers with the name Kate Connolly and 2 male passengers with the name James Kelly.
# The rows with duplicated names have multiple differences in other data, such as age & ticket number.
# These are common Irish names so it is likely that there were simply some passengers with the same first & last name.

In [13]:
# Convert the gender column into a numeric field:

In [14]:
# Checking if there are any records with missing data for sex, or any value other than 'male' or 'female' 

In [15]:
anyNA(df$sex)
# no NA's in sex column 

[1] FALSE

In [16]:
any((df$sex != 'male') & (df$sex != 'female'))

[1] FALSE

In [17]:
# Creating dummy variables for gender
df$male <- ifelse(df$sex == 'male', 1, 0)
df$female <- ifelse(df$sex == 'female', 1, 0)
head(df, 5)

pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,male,female
1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NA,"St Louis, MO",0,1
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NA,"Montreal, PQ / Chesterville, ON",1,0
1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NA,NA,"Montreal, PQ / Chesterville, ON",0,1
1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NA,135,"Montreal, PQ / Chesterville, ON",1,0
1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NA,NA,"Montreal, PQ / Chesterville, ON",0,1


In [18]:
# Create a column that gives total number of relatives on board:

In [19]:
df$relatives <- df$sibsp + df$parch
head(df, 10)

pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,male,female,relatives
1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NA,"St Louis, MO",0,1,0
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NA,"Montreal, PQ / Chesterville, ON",1,0,3
1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NA,NA,"Montreal, PQ / Chesterville, ON",0,1,3
1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NA,135,"Montreal, PQ / Chesterville, ON",1,0,3
1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NA,NA,"Montreal, PQ / Chesterville, ON",0,1,3
1,1,"Anderson, Mr. Harry",male,48.0000,0,0,19952,26.5500,E12,S,3,NA,"New York, NY",1,0,0
1,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,13502,77.9583,D7,S,10,NA,"Hudson, NY",0,1,1
1,0,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,112050,0.0000,A36,S,NA,NA,"Belfast, NI",1,0,0
1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,11769,51.4792,C101,S,D,NA,"Bayside, Queens, NY",0,1,2
1,0,"Artagaveytia, Mr. Ramon",male,71.0000,0,0,PC 17609,49.5042,NA,C,NA,22,"Montevideo, Uruguay",1,0,0


In [20]:
# Create a regression that explains which passengers survived based on the available data.

In [21]:
# Regression using female as independent variable for gender 
linearMod <- lm(survived ~ pclass + age + female + relatives, data=df)
summary(linearMod)
# Adj. R-Squared: 0.3697
# Intercept: 0.8185


Call:
lm(formula = survived ~ pclass + age + female + relatives, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.06217 -0.25457 -0.08101  0.22485  0.99096 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.8185092  0.0588974  13.897  < 2e-16 ***
pclass      -0.1840870  0.0160132 -11.496  < 2e-16 ***
age         -0.0057158  0.0009509  -6.011 2.55e-09 ***
female       0.5019920  0.0258632  19.409  < 2e-16 ***
relatives   -0.0209386  0.0087070  -2.405   0.0164 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3904 on 1041 degrees of freedom
  (263 observations deleted due to missingness)
Multiple R-squared:  0.3721,	Adjusted R-squared:  0.3697 
F-statistic: 154.2 on 4 and 1041 DF,  p-value: < 2.2e-16


In [22]:
# Try regression without intercept
linearMod <- lm(survived ~ pclass + age + female + relatives - 1, data=df)
summary(linearMod)
# Adj. R-Squared: 0.5578, increase from previous model
# P-value for pclass: 0.836, relatives: 0.393


Call:
lm(formula = survived ~ pclass + age + female + relatives - 1, 
    data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.9058 -0.1711 -0.1023  0.2607  0.9843 

Coefficients:
           Estimate Std. Error t value Pr(>|t|)    
pclass    0.0019745  0.0095604   0.207    0.836    
age       0.0045888  0.0006479   7.083 2.59e-12 ***
female    0.6068392  0.0269227  22.540  < 2e-16 ***
relatives 0.0078649  0.0092035   0.855    0.393    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4249 on 1042 degrees of freedom
  (263 observations deleted due to missingness)
Multiple R-squared:  0.5595,	Adjusted R-squared:  0.5578 
F-statistic: 330.8 on 4 and 1042 DF,  p-value: < 2.2e-16


In [23]:
# Try regression without pclass
linearMod <- lm(survived ~ age + female + relatives - 1, data=df)
summary(linearMod)
# Adj. R-Squared: 0.5582, slight increase from previous model
# P-value for relatives: 0.334


Call:
lm(formula = survived ~ age + female + relatives - 1, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.91079 -0.16846 -0.09827  0.26145  0.98960 

Coefficients:
          Estimate Std. Error t value Pr(>|t|)    
age       0.004679   0.000476   9.831   <2e-16 ***
female    0.607529   0.026703  22.752   <2e-16 ***
relatives 0.008455   0.008745   0.967    0.334    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4247 on 1043 degrees of freedom
  (263 observations deleted due to missingness)
Multiple R-squared:  0.5594,	Adjusted R-squared:  0.5582 
F-statistic: 441.5 on 3 and 1043 DF,  p-value: < 2.2e-16


In [59]:
# Try regression without relatives
linearMod <- lm(survived ~ age + female - 1, data=df)
summary(linearMod)
# Adj. R-Squared: 0.5582
# Passengers are more likely to survive if they are female, and slightly more likely to survive as age increases


Call:
lm(formula = survived ~ age + female - 1, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.9164 -0.1719 -0.0955  0.2638  0.9980 

Coefficients:
        Estimate Std. Error t value Pr(>|t|)    
age    0.0047748  0.0004657   10.25   <2e-16 ***
female 0.6156014  0.0253625   24.27   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4247 on 1044 degrees of freedom
  (263 observations deleted due to missingness)
Multiple R-squared:  0.5591,	Adjusted R-squared:  0.5582 
F-statistic: 661.8 on 2 and 1044 DF,  p-value: < 2.2e-16


In [60]:
# using the model, predict survival of 1st class 18-year old female with 0 relatives
predict(linearMod, newdata=data.frame(pclass = 1, age = 18, female = 1, relatives = 0))

1 
0.7015484

In [61]:
survival_predict <- predict(linearMod, newdata=df)
head(survival_predict,20)

1           2           3           4           5           6 
0.754071504 0.004377087 0.625151079 0.143244918 0.734972182 0.229191868 
          7           8           9          10          11          12 
0.916415744 0.186218393 0.868667438 0.339012971 0.224417037 0.701548368 
         13          14          15          16          17          18 
0.730197351 0.739747013 0.381986447          NA 0.114595934 0.854342947 
         19          20 
0.768395996 0.171893901

In [62]:
survival_predict <- na.omit(survival_predict)
head(survival_predict,20)

1           2           3           4           5           6 
0.754071504 0.004377087 0.625151079 0.143244918 0.734972182 0.229191868 
          7           8           9          10          11          12 
0.916415744 0.186218393 0.868667438 0.339012971 0.224417037 0.701548368 
         13          14          15          17          18          19 
0.730197351 0.739747013 0.381986447 0.114595934 0.854342947 0.768395996 
         20          21 
0.171893901 0.176668732

In [63]:
mean(survival_predict)

[1] 0.3710266

In [58]:
mean(df$survived)

[1] 0.381971

In [ ]:
# The model predicts 37% survival based on the data.
# The actual data has 38% survival.

In [25]:
# Regression using male as independent variable for gender 
linearMod <- lm(survived ~ pclass + age + male + relatives, data=df)
summary(linearMod)
# Adj. R-Squared: 0.3697
# Intercept: 1.3205


Call:
lm(formula = survived ~ pclass + age + male + relatives, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.06217 -0.25457 -0.08101  0.22485  0.99096 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.3205012  0.0570010  23.166  < 2e-16 ***
pclass      -0.1840870  0.0160132 -11.496  < 2e-16 ***
age         -0.0057158  0.0009509  -6.011 2.55e-09 ***
male        -0.5019920  0.0258632 -19.409  < 2e-16 ***
relatives   -0.0209386  0.0087070  -2.405   0.0164 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3904 on 1041 degrees of freedom
  (263 observations deleted due to missingness)
Multiple R-squared:  0.3721,	Adjusted R-squared:  0.3697 
F-statistic: 154.2 on 4 and 1041 DF,  p-value: < 2.2e-16


In [26]:
# Try regression without intercept
linearMod <- lm(survived ~ pclass + age + male + relatives - 1, data=df)
summary(linearMod)
# Adj. R-Squared: 0.4347, increase from previous model
# all independent variables have low P-value
# These results are different from using female as gender variable. Why?


Call:
lm(formula = survived ~ pclass + age + male + relatives - 1, 
    data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.1055 -0.2637 -0.1129  0.4607  1.1717 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
pclass     0.1062914  0.0122621   8.668  < 2e-16 ***
age        0.0111101  0.0007552  14.712  < 2e-16 ***
male      -0.4107282  0.0314529 -13.059  < 2e-16 ***
relatives  0.0441315  0.0101411   4.352 1.48e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4804 on 1042 degrees of freedom
  (263 observations deleted due to missingness)
Multiple R-squared:  0.4368,	Adjusted R-squared:  0.4347 
F-statistic: 202.1 on 4 and 1042 DF,  p-value: < 2.2e-16


In [32]:
# using the model, predict survival of 3rd class 18-year old male with 1 relative
predict(linearMod, newdata=data.frame(pclass = 3, age = 18, male = 1, relatives = 1))

1 
0.1522597

In [55]:
survival_predict <- predict(linearMod, newdata=df)
head(survival_predict,20)

1            2            3            4            5            6 
 0.428484985 -0.161857638  0.260906158  0.161261411  0.516438994  0.228849127 
           7            8            9           10           11           12 
 0.850360678  0.128858018  0.783390945  0.484381963  0.261870505  0.350405129 
          13           14           15           16           17           18 
 0.372934368  0.395154615  0.584373073           NA  0.006337669  0.705929075 
          19           20 
 0.461815355  0.095527648

In [56]:
survival_predict <- na.omit(survival_predict)
head(survival_predict,20)

1            2            3            4            5            6 
 0.428484985 -0.161857638  0.260906158  0.161261411  0.516438994  0.228849127 
           7            8            9           10           11           12 
 0.850360678  0.128858018  0.783390945  0.484381963  0.261870505  0.350405129 
          13           14           15           17           18           19 
 0.372934368  0.395154615  0.584373073  0.006337669  0.705929075  0.461815355 
          20           21 
 0.095527648  0.194900773

In [57]:
mean(survival_predict)

[1] 0.3489991

In [58]:
mean(df$survived)

[1] 0.381971

In [ ]:
# The model predicts 35% survival based on the data.
# The actual data has 38% survival.
# This model is less accurate than the model using female independent variable.
# This makes sense as the adj. R-squared for this model is smaller.

In [64]:
# Regression using male and female variables 
linearMod <- lm(survived ~ pclass + age + male + female + relatives, data=df)
summary(linearMod)
# female coefficient not defined because of singularities


Call:
lm(formula = survived ~ pclass + age + male + female + relatives, 
    data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.06217 -0.25457 -0.08101  0.22485  0.99096 

Coefficients: (1 not defined because of singularities)
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.3205012  0.0570010  23.166  < 2e-16 ***
pclass      -0.1840870  0.0160132 -11.496  < 2e-16 ***
age         -0.0057158  0.0009509  -6.011 2.55e-09 ***
male        -0.5019920  0.0258632 -19.409  < 2e-16 ***
female              NA         NA      NA       NA    
relatives   -0.0209386  0.0087070  -2.405   0.0164 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3904 on 1041 degrees of freedom
  (263 observations deleted due to missingness)
Multiple R-squared:  0.3721,	Adjusted R-squared:  0.3697 
F-statistic: 154.2 on 4 and 1041 DF,  p-value: < 2.2e-16
